In [53]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import numpy as np
import csv
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import autoreject
import scipy.fftpack
import glob

sys.path.append('../..')
from atpbar import atpbar
from utils.csv_utils import *
from utils.lsl_utils import *
from utils.MNE_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# cleaning
[link](https://autoreject.github.io/stable/auto_examples/plot_autoreject_workflow.html)


raw.filter(l_freq=1, h_freq=None)
epochs = mne.make_fixed_length_epochs(raw, duration=3, preload=True)
ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11,
                           n_jobs=1, verbose=True)
ar.fit(epochs[:20])  # fit on a few epochs to save time
epochs_ar, reject_log = ar.transform(epochs, return_log=True)

ica = mne.preprocessing.ICA(random_state=99)
ica.fit(epochs[~reject_log.bad_epochs])

# Plot Histogram

In [19]:
def plot_spectogram_bay_freq (raw, fname):
    data = raw.get_data()
    events = mne.events_from_annotations(raw)[0]

    sample_freq = 256
    window_length_s = 10
    window_length = sample_freq * window_length_s
    step_size = window_length

    # Calculate number of windows
    num_windows = (data.shape[1] - window_length) // step_size + 1

    # For storing results
    all_windows_fft = []

    # Loop over windows
    for i in range(num_windows):
        # Define start and end indices for the window
        start = i * step_size
        end = start + window_length

        # Extract window
        window = data[:, start:end]

        # Compute mean over all time series
        window_mean = np.mean(window, axis=0)

        # Compute FFT
        window_fft = np.abs(scipy.fftpack.fft(window_mean))

        # Store result
        all_windows_fft.append(window_fft)

    # Stack results
    all_windows_fft = np.stack(all_windows_fft, axis=0)

    # Keep only first half of FFT results (since they are symmetric)
    all_windows_fft = all_windows_fft[:, :window_length // 2]

    # Compute frequencies corresponding to FFT results
    frequencies = scipy.fftpack.fftfreq(window_length, d=1/sample_freq)[:window_length // 2]

    # Apply frequency mask
    mask = (frequencies >= 0) & (frequencies <= 70)
    all_windows_fft = all_windows_fft[:, mask]
    frequencies = frequencies[mask]

    # Display results
    plt.figure(figsize=(10, 6))
    plt.imshow(np.log(all_windows_fft.T), aspect='auto', cmap='hot', origin='lower',
               extent=[0, num_windows, frequencies[0], frequencies[-1]])
    #           vmin=np.min(2), vmax=np.max(-2))  # Set color bar limits here
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram (averaged over all time series)', y=1.06)
    plt.xlabel('Time (window number)')
    plt.ylabel('Frequency (Hz)')
    plt.tight_layout()
    for state in events:
        transition_time = state[0]
        state_name = state[2]
        normalized_transition_time = transition_time/256/10
        plt.axvline(x=normalized_transition_time, color='b', linestyle='--')  # Draw vertical line at each state transition
        plt.text(transition_time/data.shape[1], 1.01, state[2], transform=plt.gca().transAxes, rotation=0, va='bottom', ha='center')
    plt.tight_layout()
    plt.savefig(fname)


In [54]:
fnames = sorted(glob.glob ("/Users/jonasmago/PhD_code_data/Jhana US/analysis/mne.raw_resting_state/*-raw.fif"))
for fname in fnames:

    raw = mne.io.read_raw_fif(fname, preload=True)
    
    sfname = "/Users/jonasmago/PhD_code_data/Jhana US/analysis/mne_rest_specto/"
    sname = os.path.basename(fname)[:-8]+"_histo.png"
    spath = sfname + sname
    
    plot_spectogram_bay_freq (raw=raw, fname=spath)

Opening raw data file /Users/jonasmago/PhD_code_data/Jhana US/analysis/mne.raw_resting_state/sub0-jhana_restingstate-jhana_day1-raw.fif...
    Range : 0 ... 388359 =      0.000 ...  1517.027 secs
Ready.
Reading 0 ... 388359  =      0.000 ...  1517.027 secs...
Opening raw data file /Users/jonasmago/PhD_code_data/Jhana US/analysis/mne.raw_resting_state/sub0-jhana_restingstate-jhana_day2-raw.fif...
    Range : 0 ... 385063 =      0.000 ...  1504.152 secs
Ready.
Reading 0 ... 385063  =      0.000 ...  1504.152 secs...
Opening raw data file /Users/jonasmago/PhD_code_data/Jhana US/analysis/mne.raw_resting_state/sub0-jhana_restingstate-jhana_day3-raw.fif...
    Range : 0 ... 385231 =      0.000 ...  1504.809 secs
Ready.
Reading 0 ... 385231  =      0.000 ...  1504.809 secs...
Opening raw data file /Users/jonasmago/PhD_code_data/Jhana US/analysis/mne.raw_resting_state/sub0-jhana_restingstate-jhana_day4-raw.fif...
    Range : 0 ... 393967 =      0.000 ...  1538.934 secs
Ready.
Reading 0 ... 393

/var/folders/bq/hl737msd54q59fzf7spf0r7h0000gn/T/ipykernel_64672/2863784091.py:49: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(10, 6))


Opening raw data file /Users/jonasmago/PhD_code_data/Jhana US/analysis/mne.raw_resting_state/sub2-mindfulness_restingstate-jhana_day2-raw.fif...
    Range : 0 ... 381903 =      0.000 ...  1491.809 secs
Ready.
Reading 0 ... 381903  =      0.000 ...  1491.809 secs...
Opening raw data file /Users/jonasmago/PhD_code_data/Jhana US/analysis/mne.raw_resting_state/sub2-mindfulness_restingstate-jhana_day3-raw.fif...
    Range : 0 ... 386207 =      0.000 ...  1508.621 secs
Ready.
Reading 0 ... 386207  =      0.000 ...  1508.621 secs...
Opening raw data file /Users/jonasmago/PhD_code_data/Jhana US/analysis/mne.raw_resting_state/sub2-mindfulness_restingstate-jhana_day4-raw.fif...
    Range : 0 ... 391319 =      0.000 ...  1528.590 secs
Ready.
Reading 0 ... 391319  =      0.000 ...  1528.590 secs...
Opening raw data file /Users/jonasmago/PhD_code_data/Jhana US/analysis/mne.raw_resting_state/sub3-jhana_restingstate-jhana_day1-raw.fif...
    Range : 0 ... 384607 =      0.000 ...  1502.371 secs
Ready.

    Range : 0 ... 379575 =      0.000 ...  1482.715 secs
Ready.
Reading 0 ... 379575  =      0.000 ...  1482.715 secs...
Used Annotations descriptions: ['0', '1', '2', '3']
Opening raw data file /Users/jonasmago/PhD_code_data/Jhana US/analysis/mne.raw_resting_state/sub6-jhana_restingstate-jhana_day3-raw.fif...
    Range : 0 ... 400711 =      0.000 ...  1565.277 secs
Ready.
Reading 0 ... 400711  =      0.000 ...  1565.277 secs...
Used Annotations descriptions: ['2', '3']
Opening raw data file /Users/jonasmago/PhD_code_data/Jhana US/analysis/mne.raw_resting_state/sub6-jhana_restingstate-jhana_day4-raw.fif...
    Range : 0 ... 384343 =      0.000 ...  1501.340 secs
Ready.
Reading 0 ... 384343  =      0.000 ...  1501.340 secs...
Used Annotations descriptions: ['0', '1', '2', '3', '4']
Opening raw data file /Users/jonasmago/PhD_code_data/Jhana US/analysis/mne.raw_resting_state/sub6-mindfulness_restingstate-jhana_day1-raw.fif...
    Range : 0 ... 391023 =      0.000 ...  1527.434 secs
Ready.

    Range : 0 ... 387631 =      0.000 ...  1514.184 secs
Ready.
Reading 0 ... 387631  =      0.000 ...  1514.184 secs...
Opening raw data file /Users/jonasmago/PhD_code_data/Jhana US/analysis/mne.raw_resting_state/sub9-mindfulness_restingstate-jhana_day4-raw.fif...
    Range : 0 ... 390039 =      0.000 ...  1523.590 secs
Ready.
Reading 0 ... 390039  =      0.000 ...  1523.590 secs...
